In [20]:
!pip freeze | grep scikit-learn
!pip install pyarrow

scikit-learn==1.1.1
  Using cached pyarrow-8.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)


In [13]:
import pickle
import pandas as pd
from datetime import date

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [16]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):

    year = str(date.today().year)
    month = str(date.today().year)

    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    df['ride_id'] = year + month + df.index.astype('str')
    
    return df

In [17]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [18]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,ride_id
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,NaN,B00021,10.666667,202220221
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,NaN,B00021,14.566667,202220222
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,NaN,B00021,7.950000,202220223
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,NaN,B00037,13.800000,202220224
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,NaN,B00037,8.966667,202220225


In [10]:
df.duration.describe()

count    990113.000000
mean         16.859266
std          11.957480
min           1.000000
25%           8.000000
50%          13.833333
75%          22.466667
max          60.000000
Name: duration, dtype: float64

In [8]:
dicts = df[categorical].to_dict(orient = 'records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [23]:
output_file = "file.parquet"

df_result = df[['ride_id', 'duration']]

df_result.to_parquet(
    output_file,
    engine = 'pyarrow',
    compression = None,
    index = False
)

In [26]:
# Size parquet result
!ls -lh file.parquet

-rw-rw-r-- 1 ubuntu ubuntu 19M Jun 27 16:26 file.parquet
